In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from recomm.features import index_single_feature
from recomm.classifier import ClassifierNN, ClassifierDNN

In [2]:
init_notebook_mode(connected=True)

# Introduction

In this document, we are going to check whether the neural network is able to split a series number between 0 and 1. The simplest case is that the model must be able to distinguish the number is larger than 0.5 or not.

In [3]:
features = np.random.rand(50000)
labels = pd.get_dummies(np.where(features > 0.5, 1, 0))
features = pd.DataFrame(features)

In [4]:
features.head()

,0
0,0.454687
1,0.597216
2,0.653201
3,0.108555
4,0.022380


In [5]:
labels.head()

,0,1
0,1,0
1,0,1
2,0,1
3,1,0
4,1,0


In [6]:
train_features = features.iloc[:40000].values
train_labels = labels.iloc[:40000].values
test_features = features.iloc[40000:].values
test_labels = labels.iloc[40000:].values

In [7]:
accuracy = ClassifierNN(train_features, train_labels)\
            .build_network()\
            .set_objective(method="l2_loss")\
            .optimize()\
            .estimate(batch_size=1000, learning_rate=1e-2)\
            .predict(test_features)\
            .activate_label()\
            .get_accuracy(test_labels)\
            .accuracy

It is found the result is not as perfect as the discretized one which is discussed under feature_select_nn.ipynb.

In [8]:
accuracy

0.99629999999999996

# Effect of Multi-layers

In [10]:
hidden_layer_num = 70
hidden_layers = []
accuracy = []
for _ in range(hidden_layer_num):
    hidden_layers.append(2)
    accuracy.append(ClassifierDNN(train_features, train_labels)\
                .build_network(hidden_layers=hidden_layers)\
                .set_objective(method="l2_loss")\
                .optimize()\
                .estimate(batch_size=1000, learning_rate=1e-2)\
                .predict(test_features)\
                .activate_label()\
                .get_accuracy(test_labels)\
                .accuracy)

It is unstable after hidden layers are deeper than 50 when there are 2 units for each one.

In [11]:
data = [go.Scatter(x=np.arange(hidden_layer_num) + 1,
                   y=np.array(accuracy),
                   mode="lines",
                   name="batch_size: 50"),]
layout = go.Layout(
    title='Learning Rate: 1e-3',
    xaxis=dict(
        title='Number of Hidden Layers',
    ),
    yaxis=dict(
        title='L2 Fidelity'
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [17]:
classifier = []
for layer_num in [10, 35]:
    hidden_layers = 2 * np.ones(layer_num)
    classifier.append(ClassifierDNN(train_features, train_labels)\
        .build_network(hidden_layers=hidden_layers)\
        .set_objective(method="l2_loss")\
        .optimize()\
        .estimate(batch_size=1000, learning_rate=1e-2)\
        .predict(test_features)\
        .activate_label()\
        .get_accuracy(test_labels))

However, the pathes of convergence are different if the number of layers is an option. For example, we compare the convergence when the numbers of layers are 10 and 35. It shows the orange line approaches to asymptotic result faster but it tends to obtain the larger loss. Otherwise, the blue line convergents monotonically, and approaches to the less loss. It show it is necessary to search the various parameters when trying to optmize the loss for the different depth of networks.

In [19]:
data = [go.Scatter(x=np.arange(1000),
                   y=np.log(np.array(classifier[0].loss)),
                   mode="lines",
                   name="layer_number: 10"),
        go.Scatter(x=np.arange(1000),
                   y=np.log(np.array(classifier[1].loss)),
                   mode="lines",
                   name="layer_number: 35"),]
layout = go.Layout(
    title='Learning Rate: 1e-2',
    xaxis=dict(
        title='iteration steps',
    ),
    yaxis=dict(
        title='L2 Accuracy (Log Scale)'
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

# Add Noisy Features

In [20]:
skip_dim = 25
features_wi_noise = pd.DataFrame(features.copy())
noise_dim = np.arange(1,1000,skip_dim)
accuracy = []
for idx in noise_dim:
    features_wi_noise.loc[:, "noise_{}".format(idx)] = np.random.rand(features_wi_noise.shape[0], skip_dim)
    train_features = features_wi_noise.iloc[:40000].values
    train_labels = labels.iloc[:40000].values
    test_features = features_wi_noise.iloc[40000:].values
    test_labels = labels.iloc[40000:].values
    accuracy.append(ClassifierNN(train_features, train_labels)\
            .build_network()\
            .set_objective(method="l2_loss")\
            .optimize()\
            .estimate(batch_size=1000, learning_rate=1e-2)\
            .predict(test_features)\
            .activate_label()\
            .get_accuracy(test_labels)\
            .accuracy)

It is similar to the discrete model that the noisy features would not disturb the quality of classification when there is no hidden layers, although the accuracy is not that perfect. However, if there are too many noisy features, then the accuray would be reduced.

In [21]:
data = [go.Scatter(x=noise_dim,
                   y=np.array(accuracy),
                   mode="lines",
                   name="batch_size: 50"),]
layout = go.Layout(
    title='Learning Rate: 1e-3',
    xaxis=dict(
        title='Number of Noisy Features',
    ),
    yaxis=dict(
        title='L2 Fidelity'
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)